# 第6章 利用社交网络数据

## 获取社交网络数据的途径

- 电子邮件：通过分析用户的联系人列表，了解用户的好友信息，而且可以进一步通过研究两个用户之间的邮件往来频繁程度，度量两个用户的熟悉程度。
- 用户注册信息：需要用户在注册时填写一些诸如公司、学校等信息，获得隐性的社交网络数据。
- 用户的位置数据：通过获取用户的IP地址或者GPS数据，通过查表知道用户访问时的地址位置。
- 论坛和讨论组：如果两个用户在讨论组中曾经就某一个帖子共同进行过讨论，说明他们之间的熟悉程度或兴趣相似度很高。
- 即时聊天工具：用户在即时聊天工具上面会有一个联系人列表，通过这个列表和分组信息，知道了用户的社交网络关系。
- 社交网站：利用公开的用户社交网络和行为数据，辅助用户更好地完成信息过滤的任务，找到和自己兴趣相似的好友，找到自己感兴趣的内容。

## 社交网络数据简介

&emsp;&emsp;社交网络定义了用户之间的联系，可用图$G(V,E,w)$定义社交网络，其中$V$是顶点集合，每个顶点表示一个用户，$E$是边集合，如果用户$v_a$和$v_b$有社交关系，就有一条边$e(v_a,v_b)$连接这两个用户，而$w(v_a, v_b)$定义了边的权重。对于图$G$中的用户顶点$u$，定义out(u)为顶点$u$指向的顶点集合，定义in(u)为指向顶点$u$的集合。  
&emsp;&emsp;一般来说，有3种不同的社交网络数据:
- 双向确认的社交网络数据：通过无向图表示
- 单项关注的社交网络数据：通过有向图表示
- 基于社区的社交网络数据

## 基于社交网络的推荐

社会化推荐的优点：
- 好友推荐可以增加推荐的信任度
- 社交网络可以解决冷启动问题

### 基于领域的社会化推荐算法

&emsp;&emsp;社交网络定义了用户之间的好友关系，而用户行为数据集定义了不同用户的历史行为和兴趣数据。最简单的算法是给用户推荐好友喜欢的物品集合，即用户$u$对物品$i$的兴趣$p_{ui}$表示为$$p_{ui}=\sum_{v \in \text{out}(u)} r_{vi}$$其中，out(u)是用户$u$的好友集合，如果用户$v$喜欢物品$i$，则$r_{vi}=1$，否则$r_{vi}=0$。  
&emsp;&emsp;由于不同的好友和用户$u$的熟悉程度和兴趣相似度是不同的，需要改进兴趣计算公式：$$p_{ui}=\sum_{v \in \text{out}(u)} w_{uv} r_{vi}$$其中，$w_{uv}$由两部分相似度构成，一部分是用户$u$和用户$v$的熟悉程度，另一部分是用户$u$和用户$v$的兴趣相似度。  
&emsp;&emsp;熟悉程度可以用用户之间的共同好友比例来度量：$$\text{familiarity}(u,v)=\frac{|\text{out}(u) \cap \text{out}(v)|}{|\text{out}(u) \cup \text{out}(v)|}$$  
&emsp;&emsp;兴趣相似度可以通过和UserCF类似的方法度量，即如果两个用户喜欢的物品集合重合度很高，两个用户的兴趣相似度很高。$$\text{similiarity}(u,v)=\frac{|N(u) \cap N(v)|}{|N(u) \cup N(v)|}$$其中，$N(u)$是用户$u$喜欢的物品集合。

In [1]:
def recommend(uid, familiarity, similarity, train):
    rank = dict()
    interacted_items = train[uid]
    for fid, fw in familiarity[uid]:
        for item, pw in train[fid]:
            if item in interacted_items:
                continue
            rank[item] = fw * pw
    for vid, sw in similarity[uid]:
        for item, ow in train[fid]:
            if item in interacted_items:
                continue
            rank[item] = sw * pw
    
    return rank

### 基于图的社会化推荐算法

&emsp;&emsp;用户的社交网络可以表示为社交网络图，用户对物品的行为可以表示为用户物品的二分图，这两种图可以结合成一个图。如果用户$u$对物品$i$产生过行为，那么这两个节点之间就有边相连。  
&emsp;&emsp;用户和用户之间边的权重可以定义为用户之间相似度的$\alpha$倍（包括熟悉程度和兴趣相似度），而用户和物品之间的权重可以定义为用户对物品喜欢程度的$\beta$倍。如果希望用户好友历史行为对推荐结果影响大，则选大的$\alpha$值，如果希望用户历史行为对推荐结果影响大，则选大的$\beta$值。  
&emsp;&emsp;在定义完图中的顶点、边和边的权重后，可以利用PersonalRank图排序算法给每个用户生成推荐结果。

### 实际系统中的社会化推荐算法

基于领域的社会化推荐算法产生的问题：
1. 大型网站中用户数目和用户行为记录都非常庞大，不能将行为数据都放入缓存中。
2. 需要获得用户所有好友的历史行为数据。

Twitter的解决方案：
1. 为每个用户维护一个消息队列，用户存储他的推荐列表
2. 当一个用户喜欢一个物品时，就将（物品ID、用户ID和时间）这条记录写入关注该用户的推荐列表消息队列中
3. 当用户访问推荐系统时，读出他的推荐列表消息队列，对于这个消息队列中的每个物品，重新计算该物品的权重。计算权重时需要考虑物品在队列中出现的次数，物品对应的用户和当前用户的熟悉程度、物品的时间戳。同时，计算出每个物品被哪些好友喜欢过，用这些好友作为物品的推荐解释。

## 给用户推荐好友

### 基于内容的匹配

给用户推荐和他们有相似内容属性的用户作为好友，常见的内容属性如下：
- 用户的人口统计学属性，包括年龄、性别、职业、毕业学习和工作单元等。
- 用户的兴趣，包括用户喜欢的物品和发布过的言论等
- 用户的位置信息，包括用户的住址、IP地址和邮编等

### 基于共同兴趣的好友推荐

&emsp;&emsp;利用基于用户的协同过滤算法(UserCF)的思想，如果两个用户喜欢相同的物品（比如微博），就说明他们具有相似的兴趣；此外，也可以根据用户在社交网络中的发言提取用户的兴趣标签，来计算用户的兴趣相似度。

### 基于社交网络图的好友推荐

&emsp;&emsp;最简单的方法是给用户推荐好友的好友，主要用来给用户推荐他们在现实社会中互相熟悉，而在当前社交网络中没有联系的其他用户。

#### 基于共同好友比例的相似度计算的推荐算法（出度算法）

&emsp;&emsp;对于用户$u$和用户$v$，可以用共同好友比例计算他们的相似度：$$w_{\text{out}}(u,v)=\frac{|\text{out}(u) \cap \text{out}(v)|}{\sqrt{|\text{out}(u)||\text{out}(v)|}}$$其中，out(u)表示在社交网络图中用户$u$指向的其他好友的集合。

In [2]:
import os, sys
import pandas as pd
from sklearn.model_selection import KFold
import numpy as np

PROJECT_ROOT = os.path.dirname(sys.path[0])

In [3]:
from main.util import slashdot_reader, metric

slashdot_path = os.path.join(PROJECT_ROOT, "src/data/soc-Slashdot0902/Slashdot0902.txt")
# 加载数据集
slashdot_dataset = slashdot_reader.load_data(slashdot_path)

In [4]:
def evaluate(model, test_dataset, N):
    """
    评估模型
    :param N: 推荐的商品个数
    :param K: 搜索邻近的用户个数
    :return: 精确率(precision), 召回率(recall)
    """
    recommens = model.recommend_users(test_dataset.keys(), N=N)

    recall = metric.recall(recommends=recommens, tests=test_dataset)
    precision = metric.precision(recommends=recommens, tests=test_dataset)

    return precision, recall

In [5]:
M = 10
N = 10
kf = KFold(n_splits=M, shuffle=True, random_state=1)

index_names = []
metric_value = []

In [6]:
from main.chapter6.friend_suggestion_out import FriendSuggestionOut

precisions = []
recalls = []
# 对数据集进行划分
for train_index, test_index in kf.split(slashdot_dataset):
    test_dataset = slashdot_dataset.iloc[test_index]
    train_dataset = slashdot_dataset.iloc[train_index]
    train_dataset, test_dataset = slashdot_reader.convert_dict(train_dataset, test_dataset)
    # 模型训练
    model = FriendSuggestionOut(train_dataset)
    precision, recall = evaluate(model, test_dataset[0], N)
    precisions.append(precision)
    recalls.append(recall)
metric_value.append((np.average(precisions), np.average(recalls)))

index_names.append('FriendSuggestionOut')
pd.DataFrame(
    data=metric_value,
    index=index_names,
    columns=['Precision', 'Recall']
)

,Precision,Recall
FriendSuggestionOut,0.02139,0.016264


#### 基于共同好友比例的相似度计算的推荐算法（入度算法）

&emsp;&emsp;定义in(u)是在社交网络图中指向用户$u$的用户集合，在无向社交网络图中，out(u)和in(u)是相同的集合。可以通过in(u)定义相似度：$$w_{\text{in}}(u,v)=\frac{|\text{in}(u) \cap \text{in}(v)|}{\sqrt{|\text{in}(u)||\text{in}(v)|}}$$

In [7]:
from main.chapter6.friend_suggestion_in import FriendSuggestionIn

precisions = []
recalls = []
for train_index, test_index in kf.split(slashdot_dataset):
    test_dataset = slashdot_dataset.iloc[test_index]
    train_dataset = slashdot_dataset.iloc[train_index]
    train_dataset, test_dataset = slashdot_reader.convert_dict(train_dataset, test_dataset)
    # 模型训练
    model = FriendSuggestionIn(train_dataset)
    precision, recall = evaluate(model, test_dataset[0], N)
    precisions.append(precision)
    recalls.append(recall)
metric_value.append((np.average(precisions), np.average(recalls)))

index_names.append('FriendSuggestionIn')

pd.DataFrame(
    data=metric_value,
    index=index_names,
    columns=['Precision', 'Recall']
)

,Precision,Recall
FriendSuggestionOut,0.02139,0.016264
FriendSuggestionIn,0.00414,0.003120


#### 基于共同好友比例的相似度计算的推荐算法（出入度算法，但是不考虑热门入度用户）

&emsp;&emsp;用户$u$关注的用户中，有多大比例也关注了用户$v$，可定义如下有向的相似度：$$w_{\text{out},\text{in}}(u,v)=\frac{|\text{out}(u) \cap \text{out}(v)|}{|\text{out}(u)|}$$

In [8]:
from main.chapter6.friend_suggestion_out_in import FriendSuggestionOutIn

precisions = []
recalls = []
for train_index, test_index in kf.split(slashdot_dataset):
    test_dataset = slashdot_dataset.iloc[test_index]
    train_dataset = slashdot_dataset.iloc[train_index]
    train_dataset, test_dataset = slashdot_reader.convert_dict(train_dataset, test_dataset)
    # 模型训练
    model = FriendSuggestionOutIn(train_dataset)
    precision, recall = evaluate(model, test_dataset[0], N)
    precisions.append(precision)
    recalls.append(recall)
metric_value.append((np.average(precisions), np.average(recalls)))

index_names.append('FriendSuggestionOutIn')

pd.DataFrame(
    data=metric_value,
    index=index_names,
    columns=['Precision', 'Recall']
)

,Precision,Recall
FriendSuggestionOut,0.02139,0.016264
FriendSuggestionIn,0.00414,0.003120
FriendSuggestionOutIn,0.03986,0.030916


#### 基于共同好友比例的相似度计算的推荐算法（余弦相似度的出入度算法）

改进上面的相似度：$$w'_{\text{out},\text{in}}(u,v)=\frac{|\text{out}(u) \cap \text{in}(v)|}{\sqrt{|\text{out}(u)||\text{in}(v)|}}$$

In [9]:
from main.chapter6.friend_suggestion_out_in_cosine import FriendSuggestionOutInCosine

precisions = []
recalls = []
for train_index, test_index in kf.split(slashdot_dataset):
    test_dataset = slashdot_dataset.iloc[test_index]
    train_dataset = slashdot_dataset.iloc[train_index]
    train_dataset, test_dataset = slashdot_reader.convert_dict(train_dataset, test_dataset)
    # 模型训练
    model = FriendSuggestionOutInCosine(train_dataset)
    precision, recall = evaluate(model, test_dataset[0], N)
    precisions.append(precision)
    recalls.append(recall)
metric_value.append((np.average(precisions), np.average(recalls)))

index_names.append('FriendSuggestionOutInCosine')

pd.DataFrame(
    data=metric_value,
    index=index_names,
    columns=['Precision', 'Recall']
)

,Precision,Recall
FriendSuggestionOut,0.02139,0.016264
FriendSuggestionIn,0.00414,0.003120
FriendSuggestionOutIn,0.03986,0.030916
FriendSuggestionOutInCosine,0.01796,0.013931
